# 제주 빅데이터 경진대회

2020.07.21 화요일 김채형

**랜덤포레스트 모델링**

In [53]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import joblib

In [54]:
# 데이터 불러오기
data = pd.read_csv('/Users/kimchaehyeong/Documents/DACON/7월/modeling_data.csv', encoding='EUC-KR', index_col='Unnamed: 0')

/Users/kimchaehyeong/.pyenv/versions/anaconda3-5.3.0/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057394 entries, 1 to 1057394
Data columns (total 17 columns):
CARD_SIDO_NM                   1057394 non-null object
STD_CLSS_NM                    1057394 non-null object
HOM_SIDO_NM                    1057394 non-null object
AGE                            1057394 non-null object
SEX_CTGO_CD                    1057394 non-null int64
FLC                            1057394 non-null int64
AMT                            1057394 non-null float64
month                          1057394 non-null int64
year                           1057394 non-null int64
deceased                       1057394 non-null int64
festival_count                 1057394 non-null int64
rain_day                       1057394 non-null float64
covid                          1057394 non-null bool
age_population                 1057394 non-null float64
total_sale_by_indst_groupby    1040016 non-null float64
log_total_sale_by_indst        1040016 non-null float64
total_tour

In [56]:
data.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
1,강원,건강보조식품 소매업,강원,20s,1,1,843100.0,11,2019,0,0,8.806154,False,1.562276e+13,42767.077,10.591,1.980289e+06
2,강원,건강보조식품 소매업,강원,20s,1,1,139000.0,6,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
3,강원,건강보조식품 소매업,강원,20s,1,1,27500.0,8,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
4,강원,건강보조식품 소매업,강원,20s,1,1,395500.0,9,2019,0,11,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
5,강원,건강보조식품 소매업,강원,20s,1,1,427510.0,3,2020,1,0,7.066787,True,1.562276e+13,42767.077,10.591,NaN


In [57]:
data.shape

(1057394, 17)

### 데이터 전처리

In [58]:
# 데이터 정제
data = data.fillna('')
df = data.copy()
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [59]:
df

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,AMT,deceased,festival_count,rain_day,covid,age_population
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,311200.0,0,4,5.297286,False,1.562276e+13
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,605000.0,0,4,5.297286,False,1.562276e+13
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,139000.0,0,8,11.461487,False,1.562276e+13
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,27500.0,0,8,11.461487,False,1.562276e+13
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,395500.0,0,11,11.461487,False,1.562276e+13
5,강원,건강보조식품 소매업,강원,20s,1,1,2019,11,843100.0,0,0,8.806154,False,1.562276e+13
6,강원,건강보조식품 소매업,강원,20s,1,1,2020,1,168500.0,0,0,5.297286,False,1.562276e+13
7,강원,건강보조식품 소매업,강원,20s,1,1,2020,3,427510.0,1,0,7.066787,True,1.562276e+13
8,강원,건강보조식품 소매업,강원,20s,2,1,2019,2,912000.0,0,4,5.297286,False,1.562276e+13
9,강원,건강보조식품 소매업,강원,20s,2,1,2019,3,631000.0,0,2,7.066787,False,1.562276e+13


In [60]:
"""
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])
"""

"\n# 인코딩\ndtypes = df.dtypes\nencoders = {}\nfor column in df.columns:\n    if str(dtypes[column]) == 'object':\n        encoder = LabelEncoder()\n        encoder.fit(df[column])\n        encoders[column] = encoder\n        \ndf_num = df.copy()        \nfor column in encoders.keys():\n    encoder = encoders[column]\n    df_num[column] = encoder.transform(df[column])\n"

In [61]:
df_num = df.copy()

In [62]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid']
df_num[cat_features]= df_num[cat_features].apply(LabelEncoder().fit_transform)

In [63]:
real_cat_features =  ['SEX_CTGO_CD','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid','FLC']
for i in enumerate(real_cat_features):
    ca = i[1]
    df_num[ca] = df_num[ca].astype('category')

### 모델링

In [64]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=2020)
train_features = train_num.drop(['AMT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [65]:
# 훈련
model = RandomForestRegressor(n_jobs=-1, random_state=2020)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=2020, verbose=0, warm_start=False)

In [66]:
# save model
joblib.dump(model, 'rf_initial.pkl')
# load model
# model = joblib.load('rf_initial.pkl')

['rf_initial.pkl']

### 예측

In [67]:
# 예측 템플릿 가져오기
temp = pd.read_csv('template.csv', encoding='EUC-KR')

In [68]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,month,year,rain_day,age_population,total_sale_by_indst_groupby,total_tourist_groupby,home_ratio,covid,festival_count
0,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0
1,강원,건강보조식품 소매업,강원,20s,1,1,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0
2,강원,건강보조식품 소매업,강원,20s,1,2,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0
3,강원,건강보조식품 소매업,강원,20s,1,2,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0
4,강원,건강보조식품 소매업,강원,20s,1,3,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0
5,강원,건강보조식품 소매업,강원,20s,1,3,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0
6,강원,건강보조식품 소매업,강원,20s,1,4,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0
7,강원,건강보조식품 소매업,강원,20s,1,4,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0
8,강원,건강보조식품 소매업,강원,20s,1,5,4,2020,8.806154,15622761472137,42767.077,NaN,0.710718,True,0
9,강원,건강보조식품 소매업,강원,20s,1,5,7,2020,11.461487,15622761472137,42767.077,NaN,0.710718,True,0


In [69]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658860 entries, 0 to 1658859
Data columns (total 15 columns):
CARD_SIDO_NM                   1658860 non-null object
STD_CLSS_NM                    1658860 non-null object
HOM_SIDO_NM                    1658860 non-null object
AGE                            1658860 non-null object
SEX_CTGO_CD                    1658860 non-null int64
FLC                            1658860 non-null int64
month                          1658860 non-null int64
year                           1658860 non-null int64
rain_day                       1658860 non-null float64
age_population                 1658860 non-null int64
total_sale_by_indst_groupby    1516060 non-null float64
total_tourist_groupby          97580 non-null float64
home_ratio                     1658860 non-null float64
covid                          1658860 non-null bool
festival_count                 1658860 non-null int64
dtypes: bool(1), float64(4), int64(6), object(4)
memory usage: 178.8

In [70]:
"""
# 인코딩
dtypes = temp.dtypes
encoders = {}
for column in temp.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(temp[column])
        encoders[column] = encoder
        
temp_num = temp.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    temp_num[column] = encoder.transform(temp[column])
"""

"\n# 인코딩\ndtypes = temp.dtypes\nencoders = {}\nfor column in temp.columns:\n    if str(dtypes[column]) == 'object':\n        encoder = LabelEncoder()\n        encoder.fit(temp[column])\n        encoders[column] = encoder\n        \ntemp_num = temp.copy()        \nfor column in encoders.keys():\n    encoder = encoders[column]\n    temp_num[column] = encoder.transform(temp[column])\n"

In [71]:
# 데이터 정제
temp = temp.fillna('')
columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
temp = temp.groupby(columns).sum().reset_index(drop=False)

In [72]:
temp_num = temp.copy()

In [73]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid']
temp_num[cat_features]= temp_num[cat_features].apply(LabelEncoder().fit_transform)

In [74]:
real_cat_features =  ['SEX_CTGO_CD','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid','FLC']
for i in enumerate(real_cat_features):
    ca = i[1]
    temp_num[ca] = temp_num[ca].astype('category')

In [75]:
temp_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658860 entries, 0 to 1658859
Data columns (total 13 columns):
CARD_SIDO_NM      1658860 non-null category
STD_CLSS_NM       1658860 non-null category
HOM_SIDO_NM       1658860 non-null category
AGE               1658860 non-null category
SEX_CTGO_CD       1658860 non-null category
FLC               1658860 non-null category
year              1658860 non-null int64
month             1658860 non-null int64
rain_day          1658860 non-null float64
age_population    1658860 non-null int64
home_ratio        1658860 non-null float64
covid             1658860 non-null category
festival_count    1658860 non-null int64
dtypes: category(7), float64(2), int64(4)
memory usage: 87.0 MB


In [76]:
# 예측
pred = model.predict(temp_num)
pred = np.expm1(pred)
temp_num['AMT'] = np.round(pred, 0)
temp_num['REG_YYMM'] = temp_num['year']*100 + temp_num['month']
temp_num = temp_num[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp_num = temp_num.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [77]:
temp_num

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,4.425082e+08
1,202004,0,1,1.614507e+09
2,202004,0,2,1.189809e+09
3,202004,0,3,1.097270e+08
4,202004,0,4,9.945752e+07
5,202004,0,5,9.856097e+07
6,202004,0,6,7.257852e+08
7,202004,0,7,5.119212e+09
8,202004,0,8,3.344629e+08
9,202004,0,9,6.982268e+08


In [78]:
# 디코딩
temp_num['CARD_SIDO_NM'] = temp_num.CARD_SIDO_NM.replace(np.arange(17) , ['강원','경기','경남','경북','광주','대구','대전','부산','서울','세종','울산','인천','전남','전북','제주','충남','충북'])
temp_num['STD_CLSS_NM'] = temp_num.STD_CLSS_NM.replace(np.arange(41), df['STD_CLSS_NM'].unique())

In [79]:
temp_num

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,4.425082e+08
1,202004,강원,골프장 운영업,1.614507e+09
2,202004,강원,과실 및 채소 소매업,1.189809e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.097270e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.945752e+07
5,202004,강원,그외 기타 스포츠시설 운영업,9.856097e+07
6,202004,강원,그외 기타 종합 소매업,7.257852e+08
7,202004,강원,기타 대형 종합 소매업,5.119212e+09
8,202004,강원,기타 수상오락 서비스업,3.344629e+08
9,202004,강원,기타 외국식 음식점업,6.982268e+08


In [80]:
# id 컬럼 생성
temp_num.reset_index(level=0, inplace=True)
temp_num.rename(columns = {'index':'id'}, inplace=True)

In [81]:
temp_num

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,4.425082e+08
1,1,202004,강원,골프장 운영업,1.614507e+09
2,2,202004,강원,과실 및 채소 소매업,1.189809e+09
3,3,202004,강원,관광 민예품 및 선물용품 소매업,1.097270e+08
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,9.945752e+07
5,5,202004,강원,그외 기타 스포츠시설 운영업,9.856097e+07
6,6,202004,강원,그외 기타 종합 소매업,7.257852e+08
7,7,202004,강원,기타 대형 종합 소매업,5.119212e+09
8,8,202004,강원,기타 수상오락 서비스업,3.344629e+08
9,9,202004,강원,기타 외국식 음식점업,6.982268e+08


In [82]:
temp_num.to_csv('submission_rf_ver1.csv', index = False, encoding='utf-8')

---- 쓰레기 ----

In [67]:
x = train_num.drop(['AMT'], axis=1)
y = np.log1p(train_num['AMT'])

In [68]:
x.head(3)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,CNT,confirmed,released,deceased,festival_count,rain_day,sex_age_population,screen_num,total_tourist,total_sale_by_indst
149245,2,4,2,2,1,2,2019,4,5,5,0,0,0,14,10.7,215797,34,4.220776e+05,21073.0
554956,8,26,11,1,1,2,2020,2,3,6,73,9,0,0,5.8,708309,99,0.000000e+00,629281.0
918516,14,33,14,4,2,4,2019,5,10262,15027,0,0,0,17,10.4,52587,7,1.323866e+06,601042.0


In [69]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1057394 entries, 149245 to 305711
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   CARD_SIDO_NM         1057394 non-null  int64  
 1   STD_CLSS_NM          1057394 non-null  int64  
 2   HOM_SIDO_NM          1057394 non-null  int64  
 3   AGE                  1057394 non-null  int64  
 4   SEX_CTGO_CD          1057394 non-null  int64  
 5   FLC                  1057394 non-null  int64  
 6   year                 1057394 non-null  int64  
 7   month                1057394 non-null  int64  
 8   CSTMR_CNT            1057394 non-null  int64  
 9   CNT                  1057394 non-null  int64  
 10  confirmed            1057394 non-null  int64  
 11  released             1057394 non-null  int64  
 12  deceased             1057394 non-null  int64  
 13  festival_count       1057394 non-null  int64  
 14  rain_day             1057394 non-null  float64

In [71]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE']
for i in enumerate (cat_features) : 
    ca = i[1] 
    x[ca] = x[ca].astype('category') 

In [72]:
k = int(len(x)*0.9)

In [73]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [74]:
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_val.columns]

In [75]:
# 훈련

#train_ds = lgb.Dataset(x_train, label=y_train)
#val_ds = lgb.Dataset(x_val, label=y_val)

model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [76]:
temp_data.dtypes
obj_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE','covid']
for i in enumerate (obj_features) : 
    ca = i[1] 
    temp_data[ca] = temp_data[ca].astype('category') 

obj_features = ['year', 'month']
for i in enumerate (obj_features) : 
    ca = i[1] 
    temp_data[ca] = temp_data[ca].astype('int64') 

NameError: name 'temp_data' is not defined

In [46]:
# 훈련
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [47]:
import joblib

In [48]:
# save model
joblib.dump(model, 'lgb.pkl')

# load model
load_model = joblib.load('lgb.pkl')

In [49]:
df_num.columns = df_num.columns.tolist()

In [50]:
df_num.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD',
       'FLC', 'year', 'month', 'CSTMR_CNT', 'AMT', 'CNT', 'confirmed',
       'released', 'deceased', 'festival_count', 'rain_day',
       'sex_age_population', 'screen_num', 'total_tourist',
       'total_sale_by_indst'],
      dtype='object')

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

---- lgbm 베이스라인 코드 ----

In [ ]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [ ]:
k = int(len(x)*0.9)

In [ ]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [ ]:
import lightgbm as lgb

In [ ]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [ ]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [ ]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

In [ ]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()